In [7]:
%pip install "zenml[server]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.2 MB/s eta 0:00:00


In [8]:
!zenml integration install sklearn -y

NumExpr defaulting to 2 threads.
⠴ Installing integrations...


In [9]:
!pip install pyparsing==2.4.2

In [11]:
import IPython

IPython.Application.instance().kernel.do_shutdown(restart = True)

{'status': 'ok', 'restart': True}

In [5]:
import config
NGROK_TOKEN = config.NGROK_TOKEN

In [33]:
from zenml.environment import Environment

if Environment.in_google_colab() :
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [34]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠼ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration 
will only take effect when you're running ZenML from the initialized repository 
root, or from a subdirectory. For more information on repositories and 
configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


### ML experiment

In [35]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_test() -> None :
    digit = load_digits()
    data = digit.images.reshape((len(digit.images), -1)) # X
    X_train, X_test, y_train, y_test = train_test_split(
        data, digit.target, test_size = 0.2, shuffle = False, random_state = 42
    )
    model = SVC(gamma = 0.001)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print(f"test accuracy --> {test_acc}")

train_test()

test accuracy --> 0.9583333333333334


### Turning experiments into ML pipelines with zenML

In [36]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[Annotated[np.ndarray, "X_train"], Annotated[np.ndarray, "X_test"], Annotated[np.ndarray, "y_train"], Annotated[np.ndarray, "y_test"]] :
    digits = load_digits()
    data = digits["data"]
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits["target"], test_size = 0.2, shuffle = False, random_state = 42
    )
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(X_train: np.ndarray, y_train: np.ndarray) -> ClassifierMixin:
    model = SVC(gamma = 0.001)
    model.fit(X_train, y_train)
    return model

@step
def evaluator(X_test: np.ndarray, y_test: np.ndarray, model: ClassifierMixin) -> float :
    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print(f"test accuracy --> {test_acc}")
    return test_acc


In [37]:
from zenml import pipeline

@pipeline
def digits_pipeline() :
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train = X_train, y_train = y_train)
    evaluator(X_test = X_test, y_test = y_test, model = model)

In [38]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Reloading configuration file /content/.zen/config.yaml
Reusing registered version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of evaluator.
Step evaluator has started.
Run digits_pipeline-2023_10_05-21_06_47_564433 has finished in 2.538s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [39]:
from zenml.environment import Environment

def start_zenml_dashboar(port = 8237) :
    if Environment.in_google_colab() :
        from pyngrok import ngrok
        public_url = ngrok.connect(port)
        print(f"In colab, use this URL instead : {public_url}")
        !zenml up --blocking --port {port}

    else :
        !zenml up --port {port}

start_zenml_dashboar()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-bbdbac5d-9099-4423-975d-c69a850ec158


Opening tunnel named: http-8237-bbdbac5d-9099-4423-975d-c69a850ec158


INFO:pyngrok.process.ngrok:t=2023-10-05T21:06:52+0000 lvl=info msg="no configuration paths supplied"


t=2023-10-05T21:06:52+0000 lvl=info msg="no configuration paths supplied"


t=2023-10-05T21:06:52+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml


...

t=2023-10-05T21:13:29+0000 lvl=info msg="join connections" obj=join id=cc3892739b10 l=127.0.0.1:8237 r=103.249.39.204:55734
INFO:     103.249.39.204:0 - "GET /api/v1/info HTTP/1.1" 200 OK
INFO:     103.249.39.204:0 - "GET /api/v1/pipelines/472674d7-898f-4800-9bb5-4bf8ab7a0e6b HTTP/1.1" 200 OK
API error
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/zenml/zen_server/utils.py", line 195, in decorated
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/zenml/zen_server/routers/schedule_endpoints.py", line 86, in get_schedule
    return zen_store().get_schedule(schedule_id=schedule_id)
  File "/usr/local/lib/python3.10/dist-packages/zenml/zen_stores/sql_zen_store.py", line 3160, in get_schedule
    raise KeyError(
KeyError: "Unable to get schedule with ID '97f92da7-d5d7-4224-8b92-5fcbfb43d4fe': No schedule with this ID found."
INFO:     103.249.39.204:0 - "GET /api/v1/schedules/97f92da7-d5d7-4224-8b92-5fcbfb43d4fe HTTP/1.1" 4

INFO:pyngrok.process.ngrok:t=2023-10-05T21:13:54+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2023-10-05T21:13:54+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2023-10-05T21:13:54+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2023-10-05T21:13:54+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Finished server process [11404]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError



Use default as username and url https://1710-35-196-19-64.ngrok-free.app